In [1]:
import os
os.chdir("..")

In [2]:
import hydra
from omegaconf import DictConfig

from shell.fleet.helper import get_fleet, get_agent_cls
import torch
import time
import datetime
import logging
from shell.utils.experiment_utils import setup_experiment, process_dataset_cfg, eval_net
from shell.datasets.datasets import get_dataset
from pprint import pprint
import ray
import os
from hydra import compose, initialize
from omegaconf import OmegaConf
from shell.utils.utils import seed_everything
from shell.fleet.model_sharing_utils import is_in
logging.basicConfig(level=logging.INFO)

In [3]:
# config_path = os.path.join("conf", "grad.yaml")
# # read the config file
# cfg = omegaconf.OmegaConf.load(config_path)
initialize(config_path="conf", job_name="tmp_job")
cfg = compose(config_name="grad")
seed_everything(cfg.seed)

/tmp/ipykernel_3292290/4114385434.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf", job_name="tmp_job")
/home/vlongle/miniconda3/envs/shell/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'grad': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
# override
cfg.parallel = True

In [5]:
cfg.train.num_epochs = 50
cfg.train.component_update_freq = 50

In [6]:
AgentCls = get_agent_cls(cfg.sharing_strategy, cfg.algo, cfg.parallel)

graph, datasets, NetCls, LearnerCls, net_cfg, agent_cfg, train_cfg = setup_experiment(
        cfg)
FleetCls = get_fleet(cfg.sharing_strategy, cfg.parallel)

INFO:root:Class sequence: [4 7 0 8 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)


{'train': {'component_update_freq': 50, 'num_epochs': 50, 'save_freq': 1}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0}, 'sharing_strategy': {'name': 'gradient', 'num_coms_per_round': 5, 'retrain': {'num_epochs': 10}}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'fun', 'num_agents': 4, 'root_save_dir': 'results', 'parallel': True, 'num_init_tasks': 2, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 64, 'improvement_threshold': 0.05, 'use_contrastive': True}}


INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [2 1 3 8 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 9 7 2 1 8 6 1 6 4 5 7 8 0 2 3 0 3 9 6]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:Class sequence: [0 5 3 4 4 5 9 3 0 6 9 1 3 1 7 9 5 2 3 2]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(12

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 2, 'layer_size': 64, 'dropout': 0.0, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 2}
<class 'shell.learners.er_nocomponents.NoComponentsER'>


In [7]:
fake_dataset = get_dataset(**process_dataset_cfg(cfg))
fleet = FleetCls(graph, cfg.seed, datasets, cfg.sharing_strategy, AgentCls, NetCls=NetCls,
                     LearnerCls=LearnerCls, net_kwargs=net_cfg, agent_kwargs=agent_cfg,
                     train_kwargs=train_cfg, fake_dataset=fake_dataset)

INFO:root:Class sequence: [3 9 2 4 9 2 5 4 8 9 9 4 1 6 9 6 8 0 5 1]
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
INFO:root:(128, 1, 28, 28)
2023-03-28 14:56:05,777	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ParallelModelSyncAgent pid=3298468) INFO:root:Agent: node_id: 69420, seed: 69420000
(ParallelModelSyncAgent pid=3298468) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3298468) INFO:root:	task: 0	loss: 6.234	acc: 0.500
(ParallelModelSyncAgent pid=3298468) INFO:root:	task: avg	loss: 6.234	acc: 0.500
(ParallelModelSyncAgent pid=3298468) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3298468) INFO:root:	task: 0	loss: 6.232	acc: 0.500
(ParallelModelSyncAgent pid=3298468) INFO:root:	task: avg	loss: 6.232	acc

In [15]:
[dataset.class_sequence for dataset in datasets]

[array([4, 7, 0, 8, 4, 6, 4, 3, 8, 0, 1, 0, 5, 9, 6, 4, 9, 0, 3, 0]),
 array([2, 1, 3, 8, 6, 0, 3, 0, 5, 0, 3, 6, 2, 7, 6, 7, 6, 1, 0, 5]),
 array([0, 9, 7, 2, 1, 8, 6, 1, 6, 4, 5, 7, 8, 0, 2, 3, 0, 3, 9, 6]),
 array([0, 5, 3, 4, 4, 5, 9, 3, 0, 6, 9, 1, 3, 1, 7, 9, 5, 2, 3, 2])]

In [17]:
fake_dataset.class_sequence

array([3, 9, 2, 4, 9, 2, 5, 4, 8, 9, 9, 4, 1, 6, 9, 6, 8, 0, 5, 1])

In [8]:
def diff_models(modelA_statedict, modelB_statedict, keys=None):
    diffs = {}
    # compute the average difference between two models
    for key in modelA_statedict.keys():
        if keys is not None and not is_in(key, keys):
            continue
        if key in modelB_statedict.keys():
            diffs[key] = torch.mean(torch.abs(modelA_statedict[key] - modelB_statedict[key])).item()
    return diffs

In [9]:
# diff_models(fleet.agents[0].get_net().state_dict(), fleet.agents[1].get_net().state_dict(),
#             keys=["random_linear_projection", "components"])

print("After initialization:")
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

After initialization:


{'components.0.weight': 0.0,
 'components.0.bias': 0.0,
 'components.1.weight': 0.0,
 'components.1.bias': 0.0,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [10]:
print("Accuracy just after initialization:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Accuracy just after initialization:
{0: 0.5184079601990049}
{0: 0.5237655745269958}
{0: 0.4927099044746104}
{0: 0.5325854700854701}


In [11]:
# local train
fleet.train(0)

(ParallelModelSyncAgent pid=3358416) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3358416) INFO:root:	task: 0	loss: 6.149	acc: 0.518
(ParallelModelSyncAgent pid=3358416) INFO:root:	task: avg	loss: 6.149	acc: 0.518
(ParallelModelSyncAgent pid=3358465) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3358465) INFO:root:	task: 0	loss: 5.999	acc: 0.493
(ParallelModelSyncAgent pid=3358465) INFO:root:	task: avg	loss: 5.999	acc: 0.493
(ParallelModelSyncAgent pid=3358464) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3358464) INFO:root:	task: 0	loss: 6.157	acc: 0.524
(ParallelModelSyncAgent pid=3358464) INFO:root:	task: avg	loss: 6.157	acc: 0.524
(ParallelModelSyncAgent pid=3358571) INFO:root:epochs: 0, training task: 0
(ParallelModelSyncAgent pid=3358571) INFO:root:	task: 0	loss: 6.300	acc: 0.533
(ParallelModelSyncAgent pid=3358571) INFO:root:	task: avg	loss: 6.300	acc: 0.533
(ParallelModelSyncAgent pid=3358416) INFO:root:epochs: 1, tr

In [12]:
print("After local training:")

diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

After local training:


{'components.0.weight': 0.02265486679971218,
 'components.0.bias': 0.015560520812869072,
 'components.1.weight': 0.013678528368473053,
 'components.1.bias': 0.007566658314317465,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [13]:
print("Accuracy after local train:")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Accuracy after local train:
{0: 0.9641791044776119}
{0: 0.9667743424088602}
{0: 0.9829059829059829}
{0: 0.8520299145299145}


In [28]:
fleet.num_coms_per_round = 10
fleet.communicate(0)

(ParallelModelSyncAgent pid=3358416) INFO:root:epochs: 1, training task: 0_retrain_round_0
(ParallelModelSyncAgent pid=3358416) INFO:root:	task: 0	loss: 5.655	acc: 0.959
(ParallelModelSyncAgent pid=3358416) INFO:root:	task: avg	loss: 5.655	acc: 0.959
(ParallelModelSyncAgent pid=3358465) INFO:root:epochs: 1, training task: 0_retrain_round_0
(ParallelModelSyncAgent pid=3358465) INFO:root:	task: 0	loss: 5.500	acc: 0.943
(ParallelModelSyncAgent pid=3358465) INFO:root:	task: avg	loss: 5.500	acc: 0.943
(ParallelModelSyncAgent pid=3358464) INFO:root:epochs: 1, training task: 0_retrain_round_0
(ParallelModelSyncAgent pid=3358464) INFO:root:	task: 0	loss: 5.459	acc: 0.970
(ParallelModelSyncAgent pid=3358464) INFO:root:	task: avg	loss: 5.459	acc: 0.970
(ParallelModelSyncAgent pid=3358571) INFO:root:epochs: 1, training task: 0_retrain_round_0
(ParallelModelSyncAgent pid=3358571) INFO:root:	task: 0	loss: 5.690	acc: 0.834
(ParallelModelSyncAgent pid=3358571) INFO:root:	task: avg	loss: 5.690	acc: 0.

In [29]:
diff_models(ray.get(fleet.agents[0].get_net.remote()).state_dict(), 
            ray.get(fleet.agents[1].get_net.remote()).state_dict(),
            keys=["random_linear_projection", "components"])

{'components.0.weight': 0.004177897237241268,
 'components.0.bias': 0.0035933363251388073,
 'components.1.weight': 0.003076874651014805,
 'components.1.bias': 0.0015780297107994556,
 'random_linear_projection.weight': 0.0,
 'random_linear_projection.bias': 0.0}

In [30]:
print("Acc after sync")
# evaluate the performance again
for agent in fleet.agents:
    print(ray.get(agent.eval.remote(0)))

Acc after sync
{0: 0.9686567164179104}
{0: 0.9852330410706045}
{0: 0.9763700351935646}
{0: 0.9471153846153846}
